In [21]:
#  Maybe don't need this



import datetime
import pandas as pd
import numpy as np
import os
import csv
from collections import defaultdict
import dask.dataframe as dd

In [23]:
s3_results_dir = '/Users/brianm/Dropbox/cc_AuroraInsight/data/s3'

s3_results_file_out = 's3_scan_get4.csv'
s3_results_file_path_out = os.path.join(s3_results_dir, s3_results_file_out)

In [24]:
columns = ['col_{}'.format(x) for x in range(27) ]

In [25]:
print(columns)

['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8', 'col_9', 'col_10', 'col_11', 'col_12', 'col_13', 'col_14', 'col_15', 'col_16', 'col_17', 'col_18', 'col_19', 'col_20', 'col_21', 'col_22', 'col_23', 'col_24', 'col_25', 'col_26']


In [26]:
dtypes = {}
for column in range(27):
    dtypes['col_{0}'.format(column)] = 'object'

In [31]:
ddf = dd.read_csv(s3_results_file_path_out, 
                     dtype=dtypes,
                   )



In [32]:
ddf.compute().head()

,s3_bucket,datetime,ms,source_ip,arn,operation,s3_key,op_key,http_resp,part_size,size,sw_used,unknown_1,source_file,source_dir,prefix,file
0,aii-data,25/Sep/2020:23:16:05,0,3.236.55.190,arn:aws:sts::874847117436:assumed-role/ecs-sim...,REST.GET.OBJECT,v1/dev/terrestrial/L0/SanFrancisco/20200904/20...,GET /v1/dev/terrestrial/L0/SanFrancisco/202009...,206,16777216,3103784960,Boto3/1.14.4 Python/3.7.6 Linux/4.14.181-140.2...,-,s3_monitor2020-09-26-00-30-21-16CE7E002E59FF3C,.,v1/dev/terrestrial/L0/SanFrancisco/20200904/20...,iq_out_RX2_318EE54_0_1599407059.sigmf-data
1,aii-data,25/Sep/2020:23:16:05,0,3.236.55.190,arn:aws:sts::874847117436:assumed-role/ecs-sim...,REST.GET.OBJECT,v1/dev/terrestrial/L0/SanFrancisco/20200904/20...,GET /v1/dev/terrestrial/L0/SanFrancisco/202009...,206,16777216,3103784960,Boto3/1.14.4 Python/3.7.6 Linux/4.14.181-140.2...,-,s3_monitor2020-09-26-00-30-21-16CE7E002E59FF3C,.,v1/dev/terrestrial/L0/SanFrancisco/20200904/20...,iq_out_RX2_318EE54_0_1599407059.sigmf-data
2,aii-data,24/Sep/2020:21:16:24,0,34.207.73.67,arn:aws:sts::874847117436:assumed-role/experim...,REST.GET.OBJECT,v1/dev/terrestrial/L0/DenverPCTel/20200305/202...,GET /v1/dev/terrestrial/L0/DenverPCTel/2020030...,206,8388608,16777216,Boto3/1.15.1 Python/3.7.9 Linux/4.14.171-136.2...,-,s3_monitor2020-09-24-22-51-42-F47A5A10F82E1FE9,.,v1/dev/terrestrial/L0/DenverPCTel/20200305/202...,iq_out_RX2_318EE00_0_1583363361.sigmf-data
3,aii-data,24/Sep/2020:21:17:15,0,52.91.19.72,arn:aws:sts::874847117436:assumed-role/experim...,REST.GET.OBJECT,v1/dev/terrestrial/L0/Denver/20200917/2020-09-...,GET /v1/dev/terrestrial/L0/Denver/20200917/202...,206,8388608,3103784960,Boto3/1.15.1 Python/3.7.9 Linux/4.14.171-136.2...,-,s3_monitor2020-09-24-22-51-42-F47A5A10F82E1FE9,.,v1/dev/terrestrial/L0/Denver/20200917/2020-09-...,iq_out_RX2_3189895_0_1600275154.sigmf-data
4,aii-data,24/Sep/2020:21:17:15,0,52.91.19.72,arn:aws:sts::874847117436:assumed-role/experim...,REST.GET.OBJECT,v1/dev/terrestrial/L0/Denver/20200917/2020-09-...,GET /v1/dev/terrestrial/L0/Denver/20200917/202...,206,8388608,3103784960,Boto3/1.15.1 Python/3.7.9 Linux/4.14.171-136.2...,-,s3_monitor2020-09-24-22-51-42-F47A5A10F82E1FE9,.,v1/dev/terrestrial/L0/Denver/20200917/2020-09-...,iq_out_RX2_3189895_0_1600275154.sigmf-data


In [13]:
new_columns = [
    's3_bucket',
    'datetime',
    'ms',
    'source_ip',
    'arn',
    'operation',
    's3_key',
    'op_key',
    'http_resp',
    'part_size',
    'size',
    'sw_used',
    'unknown_1',
    'source_file',
    'source_dir'    
]

In [14]:
ddf2.columns = new_columns

In [15]:
ddf2['prefix'] = ddf2['s3_key'].map(lambda x: "/".join(x.split('/')[:-1]))

In [16]:
ddf2['file'] = ddf2['s3_key'].map(lambda x: x.split('/')[-1])

In [17]:
ddf2['datetime'] = ddf2['datetime'].map_partitions(pd.to_datetime,format='%d/%b/%Y:%H:%M:%S',meta = ('datetime64[ns]'))

In [18]:
ddf2.to_csv(os.path.join(s3_results_dir, 'get_data.csv'), single_file = True)

ValueError: time data 'aii-data' does not match format '%d/%b/%Y:%H:%M:%S' (match)